In [96]:
# %load_ext autoreload
import ujson as json
from pathlib import Path
from glob import glob
import os
from copy import deepcopy
import spacy
from spacy.gold import GoldParse, tags_to_entities
from spacy.scorer import Scorer, PRFScore
from spacy.util import load_model_from_path
from spacy.tokenizer import Tokenizer

In [220]:
model_path = "../models/plays-small-batches/epoch-834"
file_path = "../training/evaluation/fulltext-*.json"

In [221]:
LABELS = ('ACT', 'SCENE', 'OTHER-SEP', 'SPEAKER', 'M-SPEAKERS', 'HEAD-COMMENT', 'COMMENT', 
          'PAGE-NUMBER', 'FOOTER')

In [222]:
def custom_tokenizer(nlp):
    prefixes = deepcopy(nlp.Defaults.prefixes)
    prefixes += ("-",)
    
    suffixes = deepcopy(nlp.Defaults.suffixes)
    suffixes += ("-", "(?<!\.[A-Z])(?<=[A-Z])\.",)
    
    prefix_re = spacy.util.compile_prefix_regex(prefixes)
    suffix_re = spacy.util.compile_suffix_regex(suffixes)

    infix_re = spacy.util.compile_infix_regex(nlp.Defaults.infixes)
    
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                token_match=nlp.Defaults.token_match)

In [223]:
def load_file(path):
    with open(path, 'r', encoding="utf-8") as file:
        return json.loads(file.read())


def load_files(path_glob):
    files = glob(path_glob)
    data = []
    
    for filepath in files:
        data.append({
            'path': filepath,
            'json': load_file(filepath),
        })


    return data

In [224]:
nlp = load_model_from_path(model_path)
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [225]:
nlp.tokenizer = custom_tokenizer(nlp)

In [226]:
class ScorerEntity(Scorer):
    def score_entity(self, tokens, gold, ent_label, verbose=False):
        gold_ents = set(tags_to_entities([annot[-1]
                        for annot in gold.orig_annot]))
        
        if '-' not in [token[-1] for token in gold.orig_annot]:
            cand_ents = set()
            for ent in tokens.ents:
                if ent.label_ != ent_label:
                    continue
                
                first = gold.cand_to_gold[ent.start]
                last = gold.cand_to_gold[ent.end-1]
                if first is None or last is None:
                    self.ner.fp += 1
                else:
                    cand_ents.add((ent.label_, first, last))
                    
            self.ner.score_set(cand_ents, gold_ents)  
       

In [227]:
def evaluate_by_ent(text, annotations):
    label_entities = {label: [] for label in LABELS}
    scorers = {}
    
    doc = nlp.make_doc(text)
    prediction = nlp(text)
    
    for annot in annotations['entities']:
        label_entities[annot[2]].append(annot)
           
    for label in LABELS:
        scorer = ScorerEntity()

        gold = GoldParse(doc, entities=label_entities[label])

        scorer.score_entity(prediction, gold, label)

        scorers[label] = scorer
        
    return scorers



In [228]:
def print_scorers(file_path, scorers):
    print(file_path)
    print(
            "{:>12}".format("entity"),
            "{:>8}".format("ents_p"),
            "{:>8}".format("ents_r"),
            "{:>8}".format("ents_f"),
        )

    for label, scorer in scorers.items():
        print(
            "{:>12}".format(label),
            "{:>8}".format(round(scorer.scores['ents_p'], 2)),
            "{:>8}".format(round(scorer.scores['ents_r'], 2)),
            "{:>8}".format(round(scorer.scores['ents_f'], 2)),
        )

def print_global_scores(global_scores, all_scorers):
    def percent(value):
        return round(100 * value, 2)
    

    def max_of(label, name):
        return round(max(map(lambda file_score: file_score['scorer'].scores[name], all_scorers[label])), 2)
    
    
    def has_ent(scorer):
        return (scorer.ner.tp + scorer.ner.fp + scorer.ner.fn) > 0
    
    def min_of(label, name):
        scores = []
        for file_score in all_scorers[label]:
            if has_ent(file_score['scorer']) == 0:
                pass
            else:
                scores.append(file_score['scorer'].scores)
                
        return round(min(map(lambda scores: scores[name], scores)), 2)
        
    print(
            "{:<12}".format("entity"),
        
            "{:>8}".format("preci."),
            "{:>8}".format("pre min"),
            "{:>8}".format("pre max"),
        
            "{:>8}".format("recall"),
            "{:>8}".format("rec min"),
            "{:>8}".format("rec max"),
        
            "{:>8}".format("fscore"),
            "{:>8}".format("fsc min"),
            "{:>8}".format("fsc max"),
        )

    for label, score in global_scores.items():
        print(
            "{:<12}".format(label),
            
            "{:>8}".format(percent(score.precision)),
            "{:>8}".format(min_of(label, "ents_p")),
            "{:>8}".format(max_of(label, "ents_p")),
            
            "{:>8}".format(percent(score.recall)),
            "{:>8}".format(min_of(label, "ents_r")),
            "{:>8}".format(max_of(label, "ents_r")),
            
            "{:>8}".format(percent(score.fscore)),
            "{:>8}".format(min_of(label, "ents_f")),
            "{:>8}".format(max_of(label, "ents_f")),
        )
        
        f_scores = {}
        for file_score in all_scorers[label]:
            if has_ent(file_score['scorer']) == 0:
                pass
            else:
                f_scores[file_score['path']] = round(file_score['scorer'].scores['ents_f'], 2)

        min_path = min(f_scores, key=f_scores.get)
        
        if f_scores[min_path] < 100:
            print("    Worst:", min_path, f_scores[min_path], "\n")

In [229]:
file_data = load_files(file_path)
global_scores = {label: PRFScore() for label in LABELS}
all_scorers   = {label: [] for label in LABELS}

for file in file_data:
    scorers = evaluate_by_ent(*file['json'])
    
    
    for label, scorer in scorers.items():
        all_scorers[label].append({ 'path': file['path'], 'scorer': scorer })
        
        global_scores[label].tp += scorer.ner.tp
        global_scores[label].fp += scorer.ner.fp
        global_scores[label].fn += scorer.ner.fn
       

In [230]:
print_global_scores(global_scores, all_scorers)

entity         preci.  pre min  pre max   recall  rec min  rec max   fscore  fsc min  fsc max
ACT             100.0    100.0    100.0    100.0    100.0    100.0    100.0    100.0    100.0
SCENE           97.59      0.0    100.0    100.0      0.0    100.0    98.78      0.0    100.0
    Worst: ../training/evaluation/fulltext-657.json 0.0 

OTHER-SEP       44.83      0.0    100.0     50.0      0.0    100.0    47.27      0.0    100.0
    Worst: ../training/evaluation/fulltext-657.json 0.0 

SPEAKER         99.54    95.83    100.0    97.65    82.64    100.0    98.58    90.19    100.0
    Worst: ../training/evaluation/fulltext-657.json 90.19 

M-SPEAKERS      72.22      0.0    100.0    77.61      0.0    100.0    74.82      0.0    100.0
    Worst: ../training/evaluation/fulltext-1018.json 0.0 

HEAD-COMMENT    95.76    63.64    100.0    93.74    53.85    100.0    94.74    58.33    99.77
    Worst: ../training/evaluation/fulltext-1018.json 58.33 

COMMENT         67.95      0.0    100.0     73

In [ ]:
# single path evaluation :
text, annotations = load_file(file_path)
scorers = evaluate_by_ent(text, annotations)
print_scorers(file_path, scorers)